# Introduction

This notebook walks through an engagement with Wellcome Collection's catalogue API for the purpose of working with the digitised text that Wellcome makes available.

It is an end to end example, starting with accessing, cleaning and working with catalogue metadata, going through the process of acquiring the raw text of books held by Wellcome Collection and ending with the very basics of topic modelling it. In a way, it walks through an initial digital engagement to get a rough feel for the possibilities of a dataset and a digital method.

The intent of the notebook is to help in getting started at working with this text. Nothing in here is necessarily the best &mdash; or even a correct &mdash; way to do a given thing, but it will show you **a** way that can get you started.

This is an output from a collaboration with Wellcome but I do not myself work there and I do not have expert knowledge of how their catalogue works. There are surely mistakes in what I have said and done below and I am sure that there are bugs. Patches on both counts are very welcome.

See https://developers.wellcomecollection.org/docs/examples for more official and advice about working with the API.

**Using this Notebook**

This notebook is split into parts to make it easier to navigate, and to skip material of less interest. You are free to skip sections of the notebook, but you will need to run all code cells.

If you are using Jupyter then the simplest way to ensure this is to select `Restart Kernel and Run All Cells...` from the `Kernel` menu.

Be aware that outputs may change if you re-run cells without restarting the kernel. I usually notice this when later cells change variables used in earlier cells.

**Trigger Warning**

This notebook engages with historic text. Opinions expressed and language used in those books may well be insensitive or even offensive from a modern perspective. I am not aware of anything problematic in the examples that I used but some elements of the notebook are quite dynamic so I am not in control of everything that you might see.

## Setting Up

Now that we've covered what the notebook is and isn't, let's begin by installing the dependencies that we need and doing a little bit of setup.

The next cell may take a little while to run.

In [ ]:
!pip install -r requirements.txt

#You can see helper.py in the folder view on the left.
#It contains functions that this notebook uses.
#This lets us hide distracting details/complexity, but you can
#look inside it if you want to know how these things are working.
import helper

import requests
from tqdm.auto import tqdm
from jsonpath_ng.ext import parse as json_parse
from collections import Counter
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import re
import math
import random
import pandas as pd
from wordcloud import WordCloud
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim_models
from yaspin import yaspin
import ipywidgets
import pickle
import lzma
import os

from IPython.display import JSON as json_display
from IPython.core.display import Markdown

pyLDAvis.enable_notebook()
seaborn.set_theme(rc={'figure.figsize':(20,10)})
random.seed(0) #So long as this is a fixed value, operations from the random pacakge should produce the same result each time you restart the kernel.
               #Change this to None to get more random behaviour.

# Acquiring Catalogue Data

We'll start this journey by using the catalogue API to search for "typhoid".

This returns a large enough set of results to be interesting for demonstration, but a small enough set of results to be manageable.

We will call the catalogue API to begin this search, and print out what it sends back to us.

In [ ]:
#Get the first page of catalogue results
#Parameters that we pass to this function will be passed on to the catalogue API
#See https://developers.wellcomecollection.org/api/catalogue more more params
def get_cat_page(**kwargs):
  params = {
    'include': 'identifiers,subjects,production,languages,items', #items is the one that will allow us to find the text
    'pageSize': 100, #number of results per page -- 100 is the maximum
  }
  for k, v in kwargs.items():
    params[k.replace('_', '.')] = v #Wellcome API uses a some "." notation -- that won't play well with Python so we accept '_' as an alternative separator.
  response = helper.get('https://api.wellcomecollection.org/catalogue/v2/works', params) #helper.py sits alongside this notebook
  return response.json()

page = get_cat_page(query='typhoid') #query=typhoid will be passed through to the catalogue API
for key, value in page.items():
  if key != 'results': #the actual results are an overwhelming amount of output so don't show them yet
    print(f'{key}: {value}')

Here we have got one page of catalogue entries back from the API. The API returns results in pages of up to 100 results at a time.

When I wrote this cell, I got 12 pages of results for a total of 1105 catalogue entries (`totalResuts`). You may get different numbers if the collection has changed.

The actual entries are in a `results` key. Before we look at them, let's get *all* of the results. We can write a function to do that.

In [ ]:
#Given the first page of a set of catalogue results, get a complete list of pages
def get_cat_pages(page):
  if page['totalPages'] == 1:
    return [page]
  bar = tqdm(
    unit = 'pages',
    total = page['totalPages'] - 1, #-1 because we already have the first page
    desc = 'Downloading catalogue data',
  )
  pages = [page]
  while 'nextPage' in page:
    response = helper.get(page['nextPage'])
    page = response.json()
    pages.append(page)
    bar.update(1)
  return pages

#Given a list of pages, get all of the catalogue entries listed in those pages
def cat_entries(pages):
  entries = []
  for page in pages:
    entries.extend(page['results'])
  return entries

entries = cat_entries(get_cat_pages(page))

Here, `get_cat_pages` has got all of the pages of catalogue entries back for us.

We have then collected all of the entries into one list called `entries`.

So let's see what is inside a catalogue entry &mdash; watch out, there is about to be a lot of output!

In [ ]:
json_display(entries[0], expanded = True)

## Analysing Catalogue Data

That's quite a lot of data, and I believe we can get more if we keep adding to the `items` parameter up in `get_cat_page`.

We are interested in *text* about typhoid, so let's focus on the type of work that this entry describes &mdash; is it something written, or something else, like a drawing or a photograph?

The entry is in JSON format so we can use JSONPath to look this up. The JSONPath tool that we use is [jsonpath-ng](https://pypi.org/project/jsonpath-ng/) and at times we do make use of some extensions it makes to the standard JSONPath syntax.

### Formats

We'll start with the "type" of the work. One entry in the above JSON is `workType`. The `label` and `type` look relevant. Let's examine the values that these can take across the whole collection.

In [ ]:
#This cell uses dumpCount from helper.py.
#dumpCount takes a JSONPath query and a list of JSON objects
#It prints text describing query results.

print('workType types:')
helper.dumpCount('$.workType.type', entries)
print()
print('workType labels:')
helper.dumpCount('$.workType.label', entries)

I'm checking the `type` just to reassure myself about the data model. I'm not an expert in it, so I'm kind of figuring out how it works as I go. As I write this, the only `type` that I see is `Format`. This encourages me to believe that I don't need to worry about this value and can just think about the labels. If you need to be sure that you're using the data model correctly you might need to learn more about this.

`label` shows that there are a range of types of works in our results. At the time of writing, 3/4 of the works are books and several others are of types that could reasonably have text, for example "Archives and manuscripts", "Student dissertations", "E-books", "Manuscripts", "Journals".

Given that the text available through this API is provided by an OCR pipeline, it is only printed texts that are likely to have online text available. So let's filter down to just books, e-books and journals.

In [ ]:
#This filters down to a list of just books, e-books and journals.
#We cannot use JSONPath to do this because JSONPath can only check values for the
#purpose of filtering lists, and our data appears to JSONPath code as single JSON objects
#(re e.g. https://stackoverflow.com/a/43737750)
def printed_entries(entries):
  return [entry for entry in entries if entry['workType']['label'] in ['Books', 'E-books', 'Journals']]

entries = printed_entries(entries)

## Subjects

Now we've filtered down our `entries` list to cover just works that seem likely to be printed.

We might also be interested in the catalogued subjects of these works.

Working out the catalogue subject of a work is more complicated. Works in Wellcome Collection are classified according to multiple schemes. If we look in the above JSON, we can also see that the structure there is fairly complex, involving a mixture of `subject` and `concept`.

Rather than unpick all of this, let's look at a part of the structure to get a sense of how things are classified. We'll just show `subject`, and only for subjects that apply to at least 5% of the entries.

In [ ]:
print('Subjects')
#label of every member of the subjects array which has a type of Subject
helper.dumpCount('$.subjects[?(@.type=="Subject")].label', entries, 0.05)

Notice that we now have two columns of numbers.

Before, we were dealing with formats. Each catalogue entry refers to a single physical object &mdash; a book, a journal, a picture, etc etc &mdash; and so it has only one format.

Now, we are dealing with subjects and each catalogue entry may have more than one subject, so we count both "number of entries with subject out of total entries" and "number of entries with subject out of total subjects counted across all entries". Because an entry can have more than one subject it can be counted more than once.

As I write, the top line of the output reads:
`234/844 entries ( 28%);  234/2931 hits (  8%) Typhoid Fever`

This means that, out of 844 catalogue entries, 234 &mdash; or 28% of all entries &mdash; have the subject "Typhoid Fever".

Those 844 catalogue entries have a total of 2931 subjects. This includes duplicates &mdash; so if we had two catalgoue entries and they both had only the subject "Typhoid Fever", we would have still have 2 hits.

So if we list all subjects for all entries in this corpus, 234 of them &mdash; 8% of the subjects &mdash; are "Typhoid Fever".

### Deep dive into "Entries" vs "Hits"

We can unpack this with a small example. If you are not concerned about the details of this, you can fold this section away &mdash; but please run the code cells, later cells may depend on their outputs.

To begin, we can look at the subjects in a sample of ten entries. We will get ten specific IDs, so that I can write these explanations with some hope that you are seeing what I saw.



In [ ]:
#The first version of this example used the first 10 works in entries, but the catalogue might change.
#So now I get the same works by identifier, but I've changed the last two to make a better example.

#Given a list of entries and list of ids, return entries with an id in the list of ids
def get_entries_by_ids(entries, ids):
  return [entry for entry in entries if entry['id'] in ids]

#Return markdown-formatted list of some label within an entry
def dump_labels(entries, #A list of entries
                jsonpath, #Jsonpath to find a "labelled thing" (something with both the properties `label` and `id`)
                entity_name, #A friendly name for the labelled thing
                emphasis_pattern = ''): #A regular expression: anything matching it will be in bold
  searcher = json_parse(jsonpath)
  output = []
  for entry in entries:
    labelled_things = searcher.find(entry)
    output.append(f'* {entry["title"]} (id: {entry["id"]}) [**{len(labelled_things)}** {entity_name}]')
    for labelled_thing in labelled_things:
      label, id = (labelled_thing.value["label"], labelled_thing.value["id"]);
      x = f'{label} (id: {id})'
      if emphasis_pattern and re.match(emphasis_pattern, label):
        output.append(f'  * **{x}**')
      else:
        output.append(f'  *   {x}')
  return Markdown('\n'.join(output))

sample = get_entries_by_ids(entries,
                            ['bxa3fqrw','f56ccxnd','jf55amap','pw7sr9zn','q5pqqysq','qzy6ufxp','rxyt9ncw','vqhzjwd5','ab2ncfmj', 'sqwwchy7'])

display(dump_labels(sample, #our small sample of entries 
                    '$.subjects[?(@.type=="Subject")]', #jsonpath identifying the entity we are interested in
                    'subjects', #a nice name for the entity
                    '^Typhoid [Ff]ever$' #any text in the output that matches this pattern will be rendered as bold
))


What we see here is a total of 10 entries for printed works. These have varying numbers of subjects, totalling 18 (3 * 1 + 6 * 2 + 1 * 3 = 18).

I've highlighted all uses of "Typhoid Fever" as a subject. You may notice that there are two different ways of identifying the general subject of "typhoid fever" -- one spelling fever with a capital F and one with a lower case f. These two spellings also have distinct IDs. If we wanted to find all books with this general subject then we would have to use both spellings. Even then, we would have to watch out for cases like that copy of "On typhoid fever", which has both spellings.

You may also notice that there are two copies of William Thomson's "On typhoid fever". As it happens, one of these copies has two different "Typhoid fever" subjects, and one has only one of them.

Let's now run `dumpCount` over the same ten entries, first to get the titles, then to get the subjects.

In [ ]:
helper.dumpCount('$.title', sample)

"On typhoid fever" appears twice and all of the others appear once. "On typhoid fever" is therefore 20% of the sample.

The numbers here are out of 10, because there are 10 entries.

We only get one list of numbers because the number of titles equals the number of entries, so a second list would just be exactly the same as the first. `dumpCount` is written not to give us two lists when this happens.

Now let's look at subjects.

In [ ]:
helper.dumpCount('$.subjects[?(@.type=="Subject")].label', sample)

There are different numbers of entries (10) and subjects (18), as we saw above. Because of this we get two lists: the "entries" list calculates percentages as a proportion of the number of entries and the "hits" list calculates percentages as a proportion of the number of subjects.

The left-hand "entries" column is still counting "by entry" &mdash; each number is out of 10, the number of entries.

4/10 entries have the subject "Typhoid Fever", and another 3/10 have the subject "Typhoid fever" (upper-case vs lower-case "f").

Looking a few cells above, we can see that the first entry ("Typhoid fever and chronic typhoid carriers") has the subjects "Typhoid Fever - epidemiology" and "Typhoid Fever - transmission", so it effectively appears twice in the left-hand column, once for each subject. All entries will be counted once in this column for each subject that they have. Because of this, the total of entries in the left-hand column is greater than 10 &mdash; in fact it will be 18, the total number of subjects. If we count up the percentages in this column, they will come to 180% (1 * 40 + 1 * 30 + 11 * 10 = 40 + 30 + 110 = 180).

The right-hand "hits" column is counting "by subject" &mdash; each number is out of 18, the total number of subjects possessed by all of the books. Just as "On typhoid fever" appeared twice in our lists of titles, some subjects appear more than once when we list all of the subjects of all of the books. If we count up the percentages in this column, they will come to 100% (actually slightly higher because of rounding errors but it really is 18/18 if you add it up).

Note also that the inconsistent nature of the data leads to some misrepresentation. If we normalize by case, the proportions will change a little. Let's try that.

In [ ]:
#given a list of entries, returns a new list of entries with normalized subject labels
#this is rough and ready -- it normalizes label case but ignores other attributes, such as id
#Unlike most other functions in this code, this one returns clones of the original, not pointers to the original
def normalize_subjects(original_entries):
  entries = [deepcopy(entry) for entry in original_entries]
  for entry in entries:
    subjects = entry['subjects']
    seen = set()
    normalized_subjects = []
    for subject in subjects:
      if subject['type'] != 'Subject': continue #if it is not actually a subject, move on to the next subject
      lowered = subject['label'].lower()
      if lowered in seen: continue #if this work already has a subject with this label, move on to the next subject
      subject['label'] = lowered
      seen.add(lowered)
      normalized_subjects.append(subject)
    entry['subjects'] = normalized_subjects
  return entries

normalized_sample = normalize_subjects(sample) #normalized_sample is a misnomer, we are only normalizing subject label
display(dump_labels(normalized_sample, '$.subjects[?(@.type=="Subject")]', 'subjects', '^typhoid fever$'))

Our general "Typhoid Fever" subject is now consistently "typhoid fever".

We still have our same ten titles but now only 16 subjects (5 * 1 + 4 * 2 + 1 * 3 = 5 + 8 + 3 = 16) because "Typhoid fever: a history" and the first copy of "On typhoid fever" no longer have the same subject label listed twice with different cases.

We can perform the same analysis with this slightly cleaner data.

In [ ]:
helper.dumpCount('$.subjects[?(@.type=="Subject")].label', normalized_sample)

We now see easily see that 60% of the works have the "typhoid fever" subject, which is also 38% of all of the subjects covered.

There is more we could do to clean this data, and to make sure of our assumptions about the data model. For example, I am assuming that one work cannot have two completely identical subjects. But let's move on.

### Concepts

You might recall that there is more than one way of talking about classifications in the data model, so let's take a quick look at that back in our full set of entries describing printed works. We'll again look at "subjects" applying to at least 5% of books in our `entries`, but now we'll look at "concepts" too.

In [ ]:
print('Subjects')
#label of every member of the subjects array which has a type of Subject
helper.dumpCount('$.subjects[?(@.type=="Subject")].label', entries, 0.05)
print()
print('Concepts')
#label of every node at any depth beneath subjects which has a type of concept
helper.dumpCount('$.subjects..*[?(@.type=="Concept")].label', entries, 0.05)

The difference between "Typhoid Fever" and "Typhoid fever" is back with us because we only lower-cased subject labels in a sample that we copied out of `entries`. 

Still, this gives us an impression of the state of our corpus. At time of writing, 9% of these entries have no subjects and 10% have no concepts.

We can also see that the subjects and concepts are quite similar. The concepts look maybe finer-grained, but I'm mostly guessing.

If you want to see more of the subjects/concepts in the collection, make the number at the end of each `dumpCount` call smaller, or remove it to see all of them. There will be a lot.

## Works Held by Wellcome

One difficulty may be that Wellcome's catalogue includes texts belonging to other collections. These could be classified in different ways.

So let's assume that we are interested in searching works actually held by Wellcome itself and limit down to them.

The way that was suggested to me to do this was to look for works held on either open shelves or in closed stores. This seems to make sense. Perhaps it needs a tweak for purely digital works such as E-books, but at the time I was dealing with works that were old enough that I didn't need to think about that. For purposes of this notebook we will continue not to worry about that, so let's filter down our `entries`.

In [ ]:
print('All availability ids within entries:')
helper.dumpCount('$.availabilities[*].id', entries)
print()

#Given a list of entries, return those that "appear to be held by Wellcome"
def wellcome_entries(entries):
  open_searcher   = json_parse("$.availabilities[?(@.id=='open-shelves')].id")
  closed_searcher = json_parse("$.availabilities[?(@.id=='closed-stores')].id")
  return [entry for entry in entries if len(open_searcher.find(entry)) > 0 or len(closed_searcher.find(entry)) > 0]

total_entries = len(entries)
entries = wellcome_entries(entries)
print(f'{len(entries)}/{total_entries} printed works are available in closed and/or open stores (therefore held by Wellcome itself)')
total_entries = len(entries)
print('These break down as:')
helper.dumpCount('$.workType.label', entries)

We can see that `entries` uses the availabilities `online`, `closed-stores`, and `open-shelves`.

484 (at time of writing) `entries` are held by Wellcome itself, so we have replaced our old `entries` list with just these ones. Most of these entries describe books.

You may find that the total of `closed-stores` and `open-shelves` entries is greater than the number of entries held by Wellcome. I assume that this is due to some entries having copies both on open shelves and in closed stores, and/or due to multiple copies of an entry in a single location.

Now that we are looking at something like "just works held by Wellcome", we can look again at concepts and subjects to see what the coverage is like for the particular works that we are interested in.

In [ ]:
print('Subjects')
#label of every member of the subjects array which has a type of Subject
helper.dumpCount('$.subjects[?(@.type=="Subject")].label', entries, 0.05)

print()
print('Concepts')
#label of every node at any depth beneath subjects which has a type of concept
helper.dumpCount('$.subjects..*[?(@.type=="Concept")].label', entries, 0.05)

At time of writing, the subjects `Typhoid Fever - epidemiology` and `Typhoid Fever` are both well represented among printed works held at Wellcome. 30% of entries have the subject `Typhoid Fever` and 28% have the subject `Typhoid Fever - epidemiology`. Up to 58% of the whole corpus thefore has one or other of these subjects &mdash; but some works might have both subjects, so the real proportion may well be lower.

A very large 76% of entries have the concept `Typhoid Fever` at time of writing &mdash; a proportion that is even more significant given that 15% of entries have no concept at all.

As there are still entries missing subjects and concepts, lack of classification information is not explained only by a work not being held at Wellcome (or we have done something wrong).

We could keep digging, but let's just define a couple of functions for getting at subjects and concepts and then move on.

In [ ]:
#Get subject labels from an entry
def subjects(entry):
  #sorted just because I like repeatability
  return sorted(set([subject.lower() for subject in helper.list_by_jsonpath('$.subjects[?(@.type=="Subject")].label', [entry])]))

#Get concept labels from an entry
def concepts(entry):
  #sorted just because I like repeatability
  return sorted(set([concept.lower() for concept in helper.list_by_jsonpath('$.subjects..*[?(@.type=="Concept")].label', [entry])]))

print('Quick check -- subjects of first entry:')
print('\n'.join(subjects(entries[0])))

print()
print('Quick check -- concepts of first entry:')
print('\n'.join(concepts(entries[0])))

## Dates

Before we move on to looking at text, let's also get a sense of when works were published.

In [ ]:
print("Dates of printed Wellcome works, by frequency (min 1%)")
helper.dumpCount('$.production[*].dates[*].label', entries, 0.01)

Although this lists only a few dates, it is enough for us to see that the date format is not completely consistent (at least at time of writing). Sometimes we get a year, sometimes we get a year inside square brackets.

Let's see how many of the dates do not consist entirely of numbers.

In [ ]:
dates = helper.list_by_jsonpath('$.production[*].dates[*].label', entries)
not_a_number = Counter([x for x in dates if not x.isnumeric()])
print('; '.join(sorted(not_a_number.keys())))
print()
print(f'Total "bad" dates: {not_a_number.total()}/{len(dates)}')

At time of writing, there are quite a few "square brackets" cases, but also date ranges, copyright symbols, question marks and occasionally snippets of text. 28% of dates are not numbers, which seems like a lot.

Let's try just stripping out square brackets: this discards some information that might be important but for now we just want a rough sense of the date range.

In [ ]:
dates = [x.strip('[]') for x in dates]
not_a_number = Counter([x for x in dates if not x.isnumeric()])
print('; '.join(sorted(not_a_number.keys())))
print()
print(f'Total "bad" dates: {len(not_a_number)}/{len(dates)}')

At time of writing, this reduces the "bad date" proportion to 8%.

We could do more, but let's carry on with this data.

In [ ]:
print("Dates of printed Wellcome works, roughly chronologically ordered, discarding non-numbers:")
dates = sorted([int(x) for x in dates if x.isnumeric()])
print(dates)

That gives us a sense of the corpus, at time of writing the works span from 1762 to the modern day.

I also see 18 and 19. These are questionable numbers for dates. These could refer to very early works but my first guess is that these are the first two digits of a century.

Let's just drop those two numbers and then look at this data in a more visual form. We'll start by writing a function that pulls together the above rough date-handling techniques, then use that to get our dates.

In [ ]:
#Work out the date for an entry, doing some very rough cleaning
def date(entry):
  dates = helper.list_by_jsonpath('$.production[*].dates[*].label', [entry])
  if len(dates) == 0: return None
  date = dates[0] #we won't worry about dealing with more than one date
  date = date.strip('[]')
  if not date.isnumeric(): return None
  date = int(date)
  if date > 1000 and date < 2100: return date
  return None

#recompute the dates
dates = [date(entry) for entry in entries] #get date from all entries
dates = sorted([date for date in dates if date]) #sort, filtering out the Nones

Now we can generate some charts.

In [ ]:
counted_dates = Counter(dates)
total = 0
cumulative = {}
for year in set(dates):
  total += counted_dates[year]
  cumulative[year] = total

xlim = (helper.down(dates[0], 50), helper.up(dates[-1], 50))
xticks = range(xlim[0], xlim[1] + 1, 25)

ylim = (0, helper.up(counted_dates.most_common(1)[0][1], 2))
ax = seaborn.scatterplot(counted_dates)
ax.set(
  title = 'Publications per year',
  xlabel = 'Year',
  ylabel = 'Works',
  ylim = ylim,
  yticks = range(ylim[0], ylim[1] + 1, 2),
  xlim = xlim,
  xticks = xticks,
)
plt.show()

ylim = (0, helper.up(total, 50))
ax = seaborn.lineplot(cumulative)
ax.set(
  title = 'Cumulative publications per year',
  xlabel = 'Year',
  ylabel = 'Cumulative works',
  ylim = ylim,
  yticks = range(ylim[0], ylim[1] + 1, 25),
  xlim = xlim,
  xticks = xticks,
)
plt.show()

As I write, these charts show that Wellcome's own collection of printed works that are returned for a search on 'typhoid' were published mainly between the late 1800s and early 1900s -- discounting all the ones with dates that were not easy to work with.

### Languages 

OK, let's get on to actually accessing the text of our works. We'll keep the ones with funny dates, but quickly throw away the ones that are not in English, so that we're working with a single language.

We'll do this by finding all ids belonging to books that have a non-English language, and then throwing away those ids. We do it like this because some books may have more than one language listed.

In [ ]:
#Return all entries that do not have any non-English languages listed
#(Avoiding getting such texts from the API in the first place is probably a better way to deal with this)
def english_entries(entries):
  non_english_ids = set(helper.list_by_jsonpath('$.languages[?(id!="eng")].`parent`.`parent`.id', entries))
  return [entry for entry in entries if not entry['id'] in non_english_ids]

entries = english_entries(entries)
print(f'{len(entries)}/{total_entries} printed works held by Wellcome are in English (or at least are not labelled as non-English)')
total_entries = len(entries)

# Accessing Text

Now we have explored what the catalogue can tell us a little and learned a little bit about the texts in the corpus.

There is certainly more you could do to get a coherent corpus, for example looking at a particular place and/or time. But let's work with the corpus that we now have: results from a search for "typhoid", from any time, but only in English.

The next step is to find out which ones actually have digitised text available. The text is made available through a IIIF manifest, so we go looking for that.

In [ ]:
#Given a list of entries, return all those that appear to have a IIIF manifest
#TODO: If there are multiple URLs, do I get multiple entries here?
#      That would mean more than one manifest for a work, which hopefully doesn't happen.
def iiif_entries(entries):
  #This bit of JSONPath finds every member of the `items` array that ultimately turns out to contain a IIIF location.
  #It then gets the grandparent of that iten, which should be the data structure that we started with.
  #In this way, we can filter our list of printed Wellcome works to just those for which we have OCR'd text.
  return helper.list_by_jsonpath('$.items[?(@.locations[*].locationType.id="iiif-presentation")].`parent`.`parent`', entries)

entries = iiif_entries(entries)
print(f'{len(entries)}/{total_entries} entries have a IIIF manifest.')
total_entries = len(entries)

#Return a simple text description from an entry
#Very much assuming single-entry lists here
def description(entry):
  date = helper.list_by_jsonpath('$.production[*].dates[*].label', [entry])
  if len(date) > 0:
    date = date[0]
  else:
    date = '<no date>'
  return (f'id {entry["id"] if "id" in entry else "<no id>"}'
           ' -- '
          f'{entry["title"] if "title" in entry else "<no title>"}, '
          f'published {date}')

print('The first of these in the list is ' + description(entries[0]))

#Get the URL of the IIIF manifest describing an entry
def iiif_manifest_url(entry):
  return helper.expect_one(helper.list_by_jsonpath('$.items[*].locations[?(@.locationType.id="iiif-presentation")].url', [entry]))

print('Its IIIF manifest is at', iiif_manifest_url(entries[0]))

Feel free to click on that manifest and take a look. Next we will check the entries with IIIF manifests to make sure that they do actually have plain text available.

In [ ]:
#Get a IIIF manifest, given its URL
def get_iiif_manifest(url):
  return helper.get(url).json()

#Given a list of entries, return all those that appear to have OCR text actually available
def ocr_entries(entries):
  bar = tqdm(
    unit = 'works',
    total = len(entries),
    desc = 'Checking for text of works',
  )
  new_entries = []
  for entry in entries:
    try:
      manifest = get_iiif_manifest(iiif_manifest_url(entry))
      if len(helper.list_by_jsonpath('$.sequences[*].rendering[?(@.format="text/plain")].@id', [manifest])) >= 1:
        new_entries.append(entry)
    except Exception:
      print(f'Failed to get single manifest for {description(entry)}')
    bar.update(1)
  return new_entries

entries = ocr_entries(entries)
print(f'{len(entries)}/{total_entries} entries have plain text available.')
total_entries = len(entries)

In [ ]:
#Get the URL where the plain text of a work can be found
def text_url(manifest):
  return helper.expect_one(helper.list_by_jsonpath('$.sequences[*].rendering[?(@.format="text/plain")].@id', [manifest]))

print('The first entry with plain text available is', description(entries[0]))
print('Its manifest is at', iiif_manifest_url(entries[0]))
print('And the plain text is at', text_url(get_iiif_manifest(iiif_manifest_url(entries[0]))))

Again, there should only be one plain text URL and you can click on it if you would like to take a look. As a potentially large body of unformatted text, it may not be very easy to read.

Now let's get that text so that we can do something with it. We'll print out some random sentences so that we can see it has worked.

In [ ]:
#Get the plain text of a work, given the URL of that plain text
#(In other words, return the content of an arbitrary URL as plain text)
def get_plain_text(url):    
  response = helper.get(url)
  return response.text

text = get_plain_text(text_url(get_iiif_manifest(iiif_manifest_url(entries[0]))))
sentences = [x + '.' for x in text.split('.')] #an approximation to a list of sentences from the book
print("Some randomly-selected sentences, just to prove that we have got some text from a work:")
print()
print()
display(Markdown(helper.markdown_n_list(random.sample(sentences, k=10))))

Now we'll do some analysis. I'm going to use a specific id to get an example text &mdash; there's a good chance it'll be the same one we just got, but it might not be.

We'll start by getting that text, in the same way that we did above, but all in one cell this time.

In [ ]:
#Find the example that I am using
sample = helper.expect_one([entry for entry in entries if entry['id']=='kbspden9'])
print(f'The example work is {description(sample)}')

#Given an entry (rather than a URL), get that entry's plain text
def get_plain_text_from_entry(entry):
  return get_plain_text(text_url(get_iiif_manifest(iiif_manifest_url(entry))))

text = get_plain_text_from_entry(sample)
sentences = [x + '.' for x in text.split('.')] #an approximation to a list of sentences from the book

# Analyzing Text

Now we shall generate a table showing the frequencies of the most used words in the work's text, and we'll generate a word cloud showing the same information.

We will ignore common words that do not convey much information ("stop words").

In [ ]:
#If you would like to generate a cloud with different settings, adjust some of these parameters
wc = WordCloud(width=1024, height=800, min_font_size=20, 
               margin=75, #space between words
               background_color='white', color_func=lambda *args, **kwargs: 'black', #white background, black text
               max_words=10,
#               stopwords=set() #uncomment this line to prevent stopword removal, or populate the set to give your own stopwords.
                                #The default stopwords are here: https://github.com/amueller/word_cloud/blob/main/wordcloud/stopwords.
)
freq = wc.process_text(text)
wc.generate_from_frequencies(freq) #we can just to wc.generate(example_text), but this way we get the frequency table, too
display(pd.DataFrame(Counter(freq).most_common()[0:wc.max_words], columns=['word','count']).set_index('word'))
wc.to_image()

We can see that "fever" is the most frequently occuring word in the text, with "case", "one" and "â" also coming up a lot.

It might be interesting to compare the table of frequencies with the cloud. Word clouds are nice and intuitive, but do these sizes really reflect the proportions in the table? If not, how much does it matter, and does that depend on what you are using it for?

Anyway, that "â" all by itself curious. It might well be a transcription error, let's look for it in context.

In [ ]:
a_circumflex_mentions = [x for x in sentences if 'â' in x.lower()]
print(f'{len(a_circumflex_mentions)}/{len(sentences)} sentences contain "â"')
print('\nHere are the first 10 of them:\n')
display(Markdown(helper.markdown_n_list(a_circumflex_mentions[0:10])))

We can see that some of these seem are likely to be apostrophes or quote marks, as in "Budd's original essay" or "the 'Lancet,'". Others are harder to interpret. Using the URLs generated by the next cell in Wellcome's catalogue. That will give us access to a IIIF view of the work which we can use to search the text.

In [ ]:
print('Catalogue:      https://wellcomecollection.org/search/works?query=' + sample['id'])
print('Digitised work: https://wellcomecollection.org/works/' + sample['id'] + '/items')

Using the latter URL to search for some of the above sentences, I see that 'â' seems to show up as either an apostrophe/quote mark or an em-dash. So I'm going to assume it is getting rendered in place of some kinds of punctuation. Let's see if we can clean this out of our text, and what that does to the word cloud.

(I'm sure there are better ways to view the above sentences in the original text. Patches welcome.)

In [ ]:
text = text.replace('â', '')
freq = wc.process_text(text)
wc.generate_from_frequencies(freq) #we can just to wc.generate(example_text), but this way we get the frequency table, too
display(pd.DataFrame(Counter(freq).most_common()[0:10], columns=['word','count']).set_index('word'))
wc.to_image()

This has allowed "time" to make it onto our list of top words.

There is more we can do &mdash; for example, "may" seems quite meaningless unless, perhaps, it is referring to the month.

But let's move on. We'll assume that the text is clean enough with our one little "a circumflex" clean up and see wht we can with topic modelling.

# Topic Modelling

We'll use `gensim` to create a topic model of our corpus and `pyLDAVis` to visualise that.

We are working with a single text here, so we might not expect the topic model to be all that interesting, but we can see how to create and explore it.

We are getting outside of my comfort zone here &mdash; I am not an expert in topic modelling, gensim or pyLDAVis so don't take my explanations too seriously. I'm trying to point you towards how to do things, not to explain them properly.

In [ ]:
#Following https://neptune.ai/blog/pyldavis-topic-modeling-exploration-tool

#Rebuild the text into the form that gensim expects
#Don't worry too much about this bit, there is a more sensible example of setting up gensim topic models below
text = []
for word, count in freq.items():
  text.extend([word] * count)
id2word = Dictionary([text])
corpus = [id2word.doc2bow(text)]
#pprint(id2word)

lda_model = LdaModel(corpus = corpus, id2word = id2word,
                     num_topics = 3,
                     random_state = 0,
                     chunksize = 1, #Number of documents per training chunk. We actually only have 1 document.
                     alpha = 'auto',
                     eta = 'auto',
                     per_word_topics = True,
)
#doc_lda = lda_model[corpus]
#pprint(lda_model.print_topics()) #uncomment this to get a sense of what the topics "really are" (a list of words and weights)
pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)


Each circle on the left is a topic. The size of the circles indicates the extent to which that topic appears in the corpus.

The distance between topics should give a sense of how similar or different they are.

The lambda ($\lambda$) parameter at top right adjusts the weight given to term frequency vs how reflective a term is of a particular topic. Lower lambda values might make it easier to interpret the topics.

The bars on the right show frequency of word in work. If you hover over one of the circles then you'll see the word in the relevant topic as red bars. That lambda parameter plays into what you see here. Try comparing the larger circles with different lambda values, such as 0.1.

If you change the `num_topics` parameter and re-run the above cell then you will see a different model.

## Fetching the Full Corpus

Now let's collect all of the text for our corpus (printed works about typhoid held at Wellcome, not in non-English). While we are at, we will make some simpler data structures so that we can easily get at the data we're interested in.

In [ ]:
#Given a list of works, return a data structure giving easy access to some key information, including its text
def get_works(entries):
  bar = tqdm(
    unit = 'works',
    total = len(entries),
    desc = 'Downloading text of works',
  )

  works = []
  for entry in entries:
    text = ''
    try:
      text = get_plain_text_from_entry(entry)
    except Exception as e:
      print(f'Unable to get text for {description(entry)}')
      #print(f'Unable to get text for {description(entry)} [cause: {e}]')
      bar.update(1)
      continue #do not create an entry if there is no text to work with

    works.append({
      'id': entry['id'],
      'title': entry['title'],
      'date': date(entry),
      'subjects': subjects(entry),
      'concepts': concepts(entry),
      'text': text,
    })

    bar.update(1)
  return works

works = get_works(entries)
print(f'\nGot text for {len(works)}/{len(entries)} works.')

We already filtered out entries where the work had no text, so we should have got text for all of our remaining entries here.

Let's take a look at one of the data structures in our new `works` list.

In [ ]:
#Given a work, pretty-print it with markdown
def pp_work(work):
  output = []
  for key, value in works[0].items():
    key = key.capitalize()
    if key == 'Text': #just show the first 1000 characters
      output.append(f'* **{key} (first 1000 characters):** {value[0:1000]}')
    elif key == 'Tokens': #just show the first 100 tokens (will only happen if we re-run this cell after creating some tokens)
      output.append(f'* **{key} (first 100):** {value[0:100]}')
    else:
      output.append(f'* **{key}:** {value}')
  display(Markdown('\n'.join(output)))
    
pp_work(works[0])

## Preprocessing

Now we need to clean/preprocess the text. We won't get too deep into this, but we will do a few things:

1. Remove stopwords (low-information words like "and", "but")
2. Stem: Chop off certain suffixes, so that different forms of the same word are treated as the same word.
3. Tokenize: split the text into individual tokens. In this case, I believe into words.
4. Lowercase: Put all words into lower-case so that capitalized and uncapitalized words are treated as the same word.
5. Drop short tokens: Short tokens are dropped. This will, for example, get rid of single-letter words and hopefully some OCR errors.

Stemming is a bit of a blunt tool -- just chopping off certain suffixes can lead to treating different words as being the same. It can also make stop words seem to reappear, but now the "stop word" is actually a stem of some more meaningful word.
Lemmatizing is a more sophisticated alternative to stemming but it requires more compute so we will not try it here.

In [ ]:
#Given a list of works, preprocess the text in each work and store the preprocessed text in a 'tokens' key
def tokenize(works):
  bar = tqdm(
    unit = 'works',
    total = len(works),
    desc = 'Preparing text of works',
  )

  token_count = 0
  tokenized_works = []
  for work in works:
    destopped = gensim.parsing.remove_stopwords(work['text'])
    stemmed = gensim.parsing.stem_text(destopped)
    all_tokens = list(gensim.utils.tokenize(stemmed, True)) #True imposes lower case
    tokens = [token for token in all_tokens if len(token) >= 3] #Drop very short tokens
    if(len(tokens)): #drop documents that are empty after preprocessing
      work['tokens'] = tokens
      tokenized_works.append(work)
    token_count += len(tokens)
    bar.update(1)

  print(f'Generated {token_count:,} tokens')
  return tokenized_works

total_works = len(works)
works = tokenize(works)
print(f'{len(works)}/{total_works} works preprocessed for model')
total_works = len(works)

#Defined as a function just to avoid polluting the global namespace any further with short-term variables
def report_tokens(work):
  early_tokens = work['tokens'][0:100]
  print()
  print(f'First {len(early_tokens)} tokens:')
  print(' '.join(early_tokens))

  unprocessed_early_tokens = list(gensim.utils.tokenize(work['text']))[0:150]
  print()
  print(f'For comparison, the first {len(unprocessed_early_tokens)} tokens without preprocessing are:')
  print(' '.join(unprocessed_early_tokens))

report_tokens(works[0])

#Uncomment for testing the later "redownload from Wellcome" option. Typhoid corpus should be identical
#with open('original_typhoid.p', 'wb') as f:
#  pickle.dump(works, f)

## Modelling the Full Corpus

Now we will generate a new topic model from this larger corpus. The next cell may take a few minutes.

In [ ]:
#Given a list of works and an optional number of topics, generate a model with that number of topics
def model(works, num_topics = 10):
  id2word = Dictionary([work['tokens'] for work in works])
  corpus = [id2word.doc2bow(work['tokens']) for work in works]
  model = LdaModel(corpus = corpus, id2word = id2word,
                   num_topics = num_topics, #Number of topics to generate
                   random_state = 0,        #Random seed. Keeping this number the same should generate the same topics each time. Changing it should change them.
                   chunksize = len(corpus), #Number of documents per training chunk. We are dealing with a small number of docs, so may as well use all of them
                   alpha = 'auto',          #Hyperparameter -- we won't get into these
                   eta = 'auto',            #Another hyperparameter
                   per_word_topics = True,  #Generates some additional information
  )
  return id2word, corpus, model

id2word = None
corpus = None
lda_model = None

with yaspin(text="Generating topic model. May take a few minutes.", timer = True) as sp:
  id2word, corpus, lda_model = model(works)
  print(f'\nTook {int(sp.elapsed_time)}s to generate model.')


with yaspin(text='Displaying topic model'):
  display(pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word))

Once again, feel free to explore this topic model. You might again want to set $\lambda$ to a lower value.

We can also look at the topic distribution per document which &mdash; assuming that the calculation I am doing here makes sense, which it may or may not &mdash; should give a sense of which are the main topics for a particular document.

In [ ]:
#Create a simple interface
#Generate dropdown list of all available documents
#Selecting a document displays a bar graph showing importance of topic in document
def document_topics_bar():
  output2 = ipywidgets.Output()

  def doc_topics_bar(output, doc_index):
    output.clear_output()
    with(output):
      doc_topics = {'Topic #': [], 'Probability': []}
      for topic, probability in lda_model.get_document_topics(corpus[doc_index]):
        doc_topics['Topic #'].append(topic)
        doc_topics['Probability'].append(probability)
      ax = seaborn.barplot(doc_topics, x='Topic #',y='Probability')
      ax.set(
        title = f'Main topics of "{works[doc_index]["title"]}" ({works[doc_index]["id"]})',
        ylim = (0, 1),
      )
      plt.show()

  dd = ipywidgets.Dropdown(
    options = [(f'{work["title"]} ({work["date"]}) [{work["id"]}]', index) for index, work in enumerate(works)],
    value = 0,
    description = 'Work',
    layout={'width': '90%'},
  )

  def switcher(change):
    doc_topics_bar(output2, change['new'])

  switcher({'new': 0})
  dd.observe(switcher, names = 'value')
  display(dd, output2)

document_topics_bar()

Here, each bar should correspond to a topic number. This should correspond to the topic numbers in pyLDAVis.

Topics have to make a minimum contribution to a document to appear, so you generally won't see a bar for all of the topics.

## Alternative Corpora

Finally, we use the tools that we build above to create a simple interface that lets us inspect a few different corpora.

In [ ]:
#TODO: Would really like to sort out the scoping here. Setting things to global generally doesn't end well.
#TODO: Would also be good to get the bits of the UI to interact nicely.

#Define some corpora to fetch
#Note that the Wellcome API can do some filtering for us
#There are a couple of examples of that here
#See https://developers.wellcomecollection.org/api/catalogue#tag/Works/operation/getWorks for more
#Here, params is used to specify params initially passed to the Wellcome API
#max_topics is a guess as to the highest number of topics that we can cope with generating for a corpus of this size
#max_topics was set by trial and error, your mileage may vary
#max_topics limiting factor seems to be what pyLDAVis can cope with
CORPORA = {
  'Typhoid': {
    'params': {'query': 'typhoid'}, #Should result in the same corpus that we built above
    'max_topics': 100,
  },
  'Fever': {
    'params': {'query': 'fever'},
    'max_topics': 30,
  },
  '1780-1789': {
    'params': {'production.dates.from': '1780-01-01', 'production.dates.to': '1789-12-31'}, #Can do date filter direct in API
    'max_topics': 15,
  },
  '1880-1882': {
    'params': {'production.dates.from': '1880-01-01', 'production.dates.to': '1882-12-31'}, #Another date example -- narrower window as generally expect to find more works for more recent dates
    'max_topics': 10,
  },
  'Melancholy': {
    'params': {'subjects.label': 'Melancholy'}, #Can do subject filter direct in API -- but this only gets one document back, so I'm doing something wrong
    'max_topics': 100,
  },
}

#Dropdown listing available corpora
corpus_dd = ipywidgets.Dropdown(
  options = list(CORPORA.keys()),
  description = 'Corpus',
)

#Dropdown listing available views (pyLDAVis or the bar chart)
view_dd = ipywidgets.Dropdown(
  options = ['pyLDAVis', 'Document Topics'],
  value = 'Document Topics',
  description = 'View',
)
output = ipywidgets.Output()

#Generate a new topic model based on the given works and topic count
def remodel(works, topic_count):
  global id2word
  global corpus
  global lda_model
  output.clear_output()
  with yaspin(text=f"Generating topic model with {topic_count} topics. May take a few minutes.", timer = True) as sp:
    id2word, corpus, lda_model = model(works, topic_count)
    print(f'\nTook {int(sp.elapsed_time)}s to generate model.')
  output.clear_output()
  if view_dd.value == 'Document Topics':
    with output:
      document_topics_bar()
  else:
    view_dd.value = 'Document Topics'

#Get a corpus from the Wellcome API and generate a model from it
def reload(corpus, topic_count):
  global works
  def get_entries(**kwargs):
    return cat_entries(get_cat_pages(get_cat_page(**kwargs)))
  def common_filters(entries):
    return ocr_entries(english_entries(wellcome_entries(printed_entries(entries))))
  def tokens(entries):
    return tokenize(get_works(entries))

  with output:
    print('Any works with missing text will be listed as "Failed to get single manifest" or "Unable to get text", there may be quite a few...')
    print('Later progress bars will appear under this list, if nothing seems to be happening then you can scroll down to check')
    if corpus in CORPORA.keys():
      try:
        works = tokens(common_filters(get_entries(**CORPORA[corpus]['params'])))
      except Exception as e:
        print(e)
        return
    else:
      return

  with yaspin(text = 'Writing cache                                                       '):
    with lzma.open(f'{corpus}.p.xz', 'wb') as f:
      pickle.dump(works, f)

  remodel(works, topic_count)

#Get a locally-cached corpus and generate a model from it
def load(corpus, topic_count):
  global works
  with yaspin(text = 'Loading cache                                                       '):
    with lzma.open(f'{corpus}.p.xz', 'rb') as f:
      works = pickle.load(f)
  remodel(works, topic_count)

#Interface for switching between corpora
#Includes a slider for topic count and an improvised dialog box for choosing between cached corpora and reloading them from Wellcome API
def corpus_switcher(change):
  corpus = change['new']

  topic_count_slider = ipywidgets.IntSlider(
    value = 10,
    min = 2,
    max = CORPORA[corpus]['max_topics'],
    step = 1,
    description = f'# Topics (2-{CORPORA[corpus]["max_topics"]})',
    tooltip = f'Set number of topics (2-{CORPORA[corpus]["max_topics"]})',
    continuous_update = False,
    orientation = 'horizontal',
    readout = True,
    readout_format = 'd',
    layout = {'width': '75%'}
  )

  def handle_click(button):
    output.clear_output()

    #Hacky -- look at first character in button text to decide what to do
    if button.description[0] == 'L':
      load(corpus, topic_count_slider.value)
    elif button.description[0] == 'R' or button.description[0] == 'D':
      reload(corpus, topic_count_slider.value)

  output.clear_output()
  with output:
    display(topic_count_slider)
    if(os.path.isfile(f'{corpus}.p.xz')):
      loadButton   = ipywidgets.Button(description = 'Load from cache (faster)', layout = {'width': '30%'})
      reloadButton = ipywidgets.Button(description = 'Reload from Wellcome (slower)', layout = {'width': '30%'})
      loadButton.on_click(handle_click)
      reloadButton.on_click(handle_click)
      print('Cache detected: using this will save on downloading and preprocessing.')
      print('You still have to wait for the modelling.')
      display(ipywidgets.HBox([loadButton, reloadButton]))
    else:
      button = ipywidgets.Button(description = 'Download corpus and generate model', layout = {'width': '30%'})
      button.on_click(handle_click)
      display(button)

#Generate pyLDAVis interface for model currently described by the globals lda_model, corpus and id2word
def pyldavis():
  output.clear_output()
  with output:
    #The extra spaces in the yaspin text are a disgusting hack to dodge a display update bug
    with yaspin(text='Displaying topic model (may take a little while)                                         '):
      display(pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word))

#Switch between the model-viewing interfaces
def view_switcher(change):
  value = change['new']
  output.clear_output()
  with output:
    if value == 'pyLDAVis': pyldavis()
    elif value == 'Document Topics': document_topics_bar()

corpus_dd.observe(corpus_switcher, names = 'value')
view_dd.observe(view_switcher, names = 'value')
view_switcher({'new': 'Document Topics'})

display(corpus_dd, view_dd, output)

# Final Words

That's the end. Feel free to explore the models above or to hack the notebook to do your own thing.

You might notice some things that could be improved as you go. For example, a common OCR error with older text is reading "s" as "f". Or you might wish you could do something with the interface but find that you cannot.

It bears repeating that this has been a very shallow skim through applying topic modelling to this collection, designed just to give some sort of sense of the possibilities and some pointers to where to get started. Whether or not any of the topics produced here are actually useful or mean anything I cannot say but it is certainly built upon very shaky foundations.

To create useful tools or do a serious analysis with methods like these requires a good understanding of how they work, and you certainly will not get that from this notebook. But perhaps it can get you started on the journey.

# Acknowledgements

This notebook is an output from the [TNA/RLUK Professional Fellowship](https://www.rluk.ac.uk/) "Collaborative Experimentation: Research Software Prototyping for Co-Learning and Exploration in Cultural Heritage", which investigated RSE-led prototyping as a method of interdisciplinary collaboration and co-learning in digital cultural heritage.